# CIAN.ru

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from math import sin, cos, sqrt, radians

In [2]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

**Словарь, содержащий информацию о квартирах Москвы**

In [3]:
#Создание словаря с клчючом "District". По этому ключу будет хранится список,  
flatStats = {'District':[]} #инициализация списка, хранящего сведения о квартирах

**Введём численные обозначения округов Москвы**:
* 0 - ЦАО
* 1 - САО
* 2 - СВАО
* 3 - ВАО
* 4 - ЮВАО
* 5 - ЮАО
* 6 - ЮЗАО
* 7 - ЗАО
* 8 - СЗАО
* 9 - НАО
* 10 - ТАО

# Описание парсинга для ЦАО, для остальных округов действия будут аналогичными.
### Для ЦАО находим главную ссылку и в ней находим **&p=1&**, это и есть переход по страницам.

In [4]:
district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

In [5]:
district

'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

### Шагаем по страницам и качаем с них все ссылки на квартиры

In [6]:
links = []
for page in range(1, 30):
    page_url =  district.format(page)

    search_page = requests.get(page_url)
    search_page = search_page.content
    search_page = BeautifulSoup(search_page, 'lxml')

    flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
    flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

    for link in flat_urls:
        if link.isdigit():
            links.append(link)

In [ ]:
links

In [8]:
len(links)

812

Итого вытащили 812 квартир по ЦАО

### Переход на первую из них

In [9]:
flat_url = 'http://www.cian.ru/sale/flat/' + str(links[0]) + '/'
#flat_url = 'http://www.cian.ru/sale/flat/150531912/' 
flat_page = requests.get(flat_url)
flat_page = flat_page.content
flat_page = BeautifulSoup(flat_page, 'lxml')

In [10]:
flatStats['District']+=[0] #работаем с ЦАО

# Цена

In [11]:
price = flat_page.find('div', attrs={'class':'object_descr_price'})
price = re.split('<div>|за м<sup>2</sup>|\n|руб|\W', str(price))

In [12]:
print(price)

['', 'div', 'class', '', 'object_descr_price', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '55', '000', '000', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'div', '']


In [13]:
[i for i in price if i.isdigit()][-3:]

['55', '000', '000']

In [14]:
"".join([i for i in price if i.isdigit()][-3:])

'55000000'

In [15]:
price = "".join([i for i in price if i.isdigit()][-3:])

In [16]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    try:
        return int(price)
    except:
        return '-'

In [17]:
getPrice(flat_page)

55000000

In [18]:
flatStats['Price'] = [] #список под хранение цен на квартиры
flatStats['Price'] += [getPrice(flat_page)]

# Координаты местонахождения квартир

In [19]:
flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]

<a href="http://map.cian.ru/?deal_type=2&amp;flats=yes&amp;center=55.759208%2C37.643408&amp;room0=1&amp;room1=1&amp;room2=1&amp;room3=1&amp;room4=1" rel="nofollow" target="_blank"><i></i>Посмотреть предложения рядом</a>

In [20]:
coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
coords = re.split('&amp|center=|%2C', str(coords))
coords_list = []
for item in coords:
    if item[0].isdigit():
        coords_list.append(item)
lat = float(coords_list[0])
lon = float(coords_list[1])

In [21]:
lat, lon

(55.759208, 37.643408)

In [22]:
def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0]) #широта
    lon = float(coords_list[1]) #долгота
    return lat, lon

In [23]:
getCoords(flat_page)

(55.759208, 37.643408)

# Расстояние от квартиры до центра Москвы(в км)

**Положим, что центром Москвы является Красная площадь**.

Её координаты:

широта: **55.753594**;

долгота: **37.622601**.

Так как площадь поверхности, занимаемой Москвой, мала по сравнению с площадью поверхности Земли, то при расчёте расстояний от квартиры до Красной площади кривизной поверхности можно пренебречь. Т.е. поверхность, занимаемая Москвой, -- это плоскость.


In [24]:
#Радиус Земли, в км 
R = 6371.0
#Географические координаты Красной Площади
lat_RS = 55.753594
lon_RS = 37.622601

Географические координаты тесно связаны со сферическими координатами, а сферические -- с декартовыми.

Переход от географических координат к декартовым координатам:

In [25]:
x = R*sin(radians(lat))*cos(radians(lon))
y = R*sin(radians(lat))*sin(radians(lon))
#координату z не рассматриваем в силу предположения о не кривизне поверхности  

In [26]:
#Декартовы координаты Красной площади
x_RS = R*sin(radians(lat_RS))*cos(radians(lon_RS))
y_RS = R*sin(radians(lat_RS))*sin(radians(lon_RS))

In [27]:
#искомое расстояние, в км
distance = sqrt(((x - x_RS)**2) + ((y - y_RS)**2))

In [28]:
distance

1.9445613162783342

In [29]:
def Dist(flat_page):
    #Радиус Земли, в км 
    R = 6371.0
    #Географические координаты Красной Площади
    lat_RS = 55.753594
    lon_RS = 37.622601
    
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0]) 
    lon = float(coords_list[1]) 
    
    x = R*sin(radians(lat))*cos(radians(lon))
    y = R*sin(radians(lat))*sin(radians(lon))
    x_RS = R*sin(radians(lat_RS))*cos(radians(lon_RS))
    y_RS = R*sin(radians(lat_RS))*sin(radians(lon_RS))
    
    return  float("{0:.2f}".format(sqrt(((x - x_RS)**2) + ((y - y_RS)**2)))) #<--- возвращается расстояние, округлённое до двух знаков после запятой
    

In [30]:
Dist(flat_page)

1.94

In [31]:
#Список под хранение расстояний до центра
flatStats['Dist'] = [] 
flatStats['Dist'] += [Dist(flat_page)]

# Число комнат

In [32]:
rooms = flat_page.find('div', attrs={'class':'object_descr_title'})

In [33]:
rooms

<div class="object_descr_title" itemprop="name">
                
    
        4-комн. кв. 
    

                
                    
                
                </div>

In [34]:
rooms = html_stripper(rooms)

In [35]:
rooms

'\n                \n    \n        4-комн. кв.\xa0\n    \n\n                \n                    \n                \n                '

In [36]:
room_number = ''
for i in re.split('-|\n', rooms):
    if 'комн' in i:
        break
    else:
        room_number += i
room_number = "".join(room_number.split())

In [37]:
room_number

'4'

In [38]:
def getRoom(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    room_number = "".join(room_number.split())
    try:
        return int(room_number)
    except:
        return '-' #в случае, если на странице не оказалась информация о числе комнат в квартире

In [39]:
getRoom(flat_page)

4

In [40]:
#Список под хранение числа комнат
flatStats['Rooms'] = [] 
flatStats['Rooms'] += [getRoom(flat_page)]

# Расстояние до метро в минутах

In [41]:
metrdist = flat_page.find('span', attrs={'class':'object_item_metro_comment'})

In [42]:
metrdist

<span class="object_item_metro_comment">
                10
                мин.
                
                    пешком
                
            </span>

In [43]:
metrdist = html_stripper(metrdist)

In [44]:
metrdist

'\n                10\n                мин.\n                \n                    пешком\n                \n            '

In [45]:
#разбиение строки metrdist
metrdist.split()

['10', 'мин.', 'пешком']

In [46]:
#Ищем расстояние до метро
for i in metrdist.split():
    if i.isdigit():
        print(int(i))

10


In [47]:
def Metrdist(flat_page):
    metrdist = flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    metrdist = html_stripper(metrdist)
    for i in metrdist.split():
        if i.isdigit():
            return int(i)
        else:
            return '-' #в случае, если отсутствует информация о времени

In [48]:
Metrdist(flat_page)

10

In [49]:
#Список под хранение расстояния до метро в минутах.
flatStats['Metrdist'] = [] 
flatStats['Metrdist'] += [Metrdist(flat_page)]

# Расстояние до метро, если пешком

In [50]:
def Walk(flat_page):
    walk = flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    walk = html_stripper(walk)
    if "пешком" in walk:
        return 1
    else:
        return 0

In [51]:
Walk(flat_page)

1

In [52]:
#Список под хранение результата предиката [расстояния до метро,если пешком].
flatStats['Walk'] = [] 
flatStats['Walk'] += [Walk(flat_page)]

# Общая информация о квартире

In [53]:
table = flat_page.find('table', attrs = {'class':'object_descr_props'})

In [54]:
table = html_stripper(table)

In [ ]:
print(html_stripper(table)) #регулярные выражения для очистки от HTML-кода нам в помощь

### Номер этажа, на котором расположена квартира

In [ ]:
#Разбиение table по пробелам
table.split()

In [57]:
#Ищем слово "Этаж" в общей информации о квартире
for i in range(len(table.split())):
    if "Этаж" in table.split()[i]: 
        print(table.split()[i+1]) 

2


In [58]:
#Объявление и определение функции, возвращающей номер этажа, на котором расположена квартира 
def getFloor(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    for i in range(len(table.split())):
        if "Этаж:" in table.split()[i]: 
            return int(table.split()[i+1])  #следующий элемент в списке после "Этаж" -- это номер этажа, на котором находится квартира
    
    #вдруг оказалось так, что нет информации об этаже
    if "Этаж:" not in table.split():
        return '-'

In [59]:
getFloor(flat_page)

2

In [60]:
#Список под хранение номера этажа, на котором расположена квартира
flatStats['Floor'] = []
flatStats['Floor'] +=[getFloor(flat_page)] 

### Всего этажей в доме

In [61]:
#Объявление и определение функции, возвращающей общее количество этажей в доме 
def getNfloors(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    for i in range(len(table.split())):
        if "Этаж:" in table.split()[i]: 
            try:
                return int(table.split()[i+3])  #третий элемент в списке после "Этаж" -- это общее количество этажей в доме
            except:
                return '-'
    #вдруг оказалось так, что нет информации об этаже или вообще отсутствует такое поле
    if "Этаж:" not in table.split():
        return '-'

In [62]:
getNfloors(flat_page)

6

In [63]:
#Список под хранение количество этажей в доме
flatStats['Nfloors'] = []
flatStats['Nfloors'] +=[getNfloors(flat_page)] 

### Тип дома: первичный рынок - 1,  вторичка - 0 

In [64]:
#Проверяем, в каком рынке находится дом
if "вторичка" in table:
    print(1)
else:
    print(0)

1


In [65]:
def is_New(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    if "вторичка" in table:
        return 1
    else:
        return 0

In [66]:
is_New(flat_page)

1

In [67]:
#Список под хранение информации о том, в каком рынке находится дом
flatStats['New'] = []
flatStats['New'] += [is_New(flat_page)]

### Балкон/лоджия

In [ ]:
table.split()

In [69]:
#Функция, отвечающая на вопрос: есть ли в доме балкон/лоджия?
def is_there_Bal(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    for i in range(len(table.split())):
        if "Балкон:" in table.split()[i]: 
            try:
                int(table.split()[i+1])
                return 1
            except:
                return 0
    #вдруг оказалось так, что нет информации об этаже или вообще отсутствует такое поле
    if "Балкон:" not in table.split():
        return '-'

In [70]:
is_there_Bal(flat_page)

0

In [71]:
#Cписок под хранение информации о наличии балконов/лоджей  
flatStats['Bal'] = []
flatStats['Bal'] += [is_there_Bal(flat_page)]

### Наличие домашнего телефона в доме

In [72]:
def is_there_Tel(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    for i in range(len(table.split())):
        if "Телефон:" in table.split()[i]: 
            if table.split()[i+1]=='да':
                return 1
            else:
                return 0
    #вдруг оказалось так, что нет информации об этаже или вообще отсутствует такое поле
    if "Телефон:" not in table.split():
        return '-'

In [73]:
is_there_Tel(flat_page)

1

In [74]:
#Cписок под хранение информации о наличии телефона
flatStats['Tel'] = []
flatStats['Tel'] += [is_there_Tel(flat_page)]

### Тип дома

In [75]:
def is_Brick(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    if "кирпичный" in table:
        return 1
    elif "монолитный" in table:
        return 1
    elif "железобетонный" in table:
        return 1
    else:
        return 0 

In [76]:
is_Brick(flat_page)

1

In [77]:
#Cписок под хранение информации о типе дома
flatStats['Brick'] = []
flatStats['Brick'] += [is_Brick(flat_page)]

### Общая площадь квартиры

In [78]:
#Функция, определяющая общую площадь комнаты
def Totsp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    #Разбиение table по пробелам
    table = table.split()
    
    #Поиск общей площади
    for i in range(len(table)):
        if 'Общая' == table[i]:
            if 'площадь:' == table[i+1]:
                area_total_not_num = table[i+2] #площадь в не числовом формате!
                #Преобразование общей площади к float
                if ',' in area_total_not_num:
                    area_total_not_num  = area_total_not_num.replace(',','.')
                    try:
                        return float(area_total_not_num)
                    except:
                        return '-'  #вдруг, если какой-то баг
                try:
                    return float(area_total_not_num)
                except:
                    return '-'  #вдруг, если отсутствует информация

In [79]:
Totsp(flat_page)

152.0

In [ ]:
table.split()

In [81]:
#Cписок под хранение общей площади
flatStats['Totsp'] = []
flatStats['Totsp'] += [Totsp(flat_page)]

### Жилая площадь квартиры

In [ ]:
re.split('Площадь|:|м2', table)

In [83]:
#Разбиение строки по 'Площадь', ':', 'м2'
re.split('Площадь|:|м2', table)

#Ищем "Жилая площадь" в разбиении, чтобы выйти на жилую площадь 
for i in range(len(re.split('Площадь|:|м2', table))):
    if "Жилая площадь" in re.split('Площадь|:|м2', table)[i]:
        area_live_not_num = re.split('Площадь|:|м2', table)[i+1] #жилая площадь в не числовом формате

In [84]:
area_live_not_num 

'\n\n100\xa0'

In [85]:
#Преобразование жилой площади к float
if ',' in area_live_not_num:
    area_live_not_num = area_live_not_num.replace(',','.')
try:
    area_live = float(area_live_not_num)
except:
    area_live = '-'

In [86]:
area_live


100.0

In [87]:
#Автоматизируем приведённые выше действия
def Livesp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    
    for i in range(len(re.split('Площадь|:|м2', table))):
        if "Жилая площадь" in re.split('Площадь|:|м2', table)[i]:
            area_live_not_num = re.split('Площадь|:|м2', table)[i+1]
    
            if ',' in area_live_not_num:
                area_live_not_num = area_live_not_num.replace(',','.')
                try:
                    return float(area_live_not_num)
                except:
                    return '-'
            try:
                return float(area_live_not_num)
            except:
                return '-'

In [88]:
Livesp(flat_page)

100.0

In [89]:
#Cписок под хранение жилой площади
flatStats['Livesp'] = []
flatStats['Livesp'] += [Livesp(flat_page)]

### Площадь кухни 

In [90]:
#Разбиение строки по 'площадь', ':', 'м2'
fragmentation = re.split('Площадь|:|м2', table)

In [91]:
print(fragmentation)

['\n\n # Общая информация', '\n\n\nЭтаж', '\n\n                2\xa0/\xa06\n                \n            \n\n\nТип дома', '\n\n            вторичка, \n            кирпичный\n        \n\n\nТип продажи', '\nсвободная\n\n\nОбщая площадь', '\n\n152\xa0', '\n\n\n\n', ' комнат', '\n\n49-17-17-10\xa0', '\n\n\n\nЖилая площадь', '\n\n100\xa0', '\n\n\n\n', ' кухни', '\n\n17\xa0', '\n\n\n\nРаздельных санузлов', '\n1\n            \n\n\nСовмещенных санузлов', '\n1\n            \n\n\nБалкон', '\n–\n        \n\n\nЛифт', '\n1 пасс.\n\n\nТелефон', '\nда\n\n\nВид из окна', '\nдвор и улица\n\n\nРемонт', '\nдизайнерский\n\n']


In [92]:
#Ищем слово "кухни" в fragmentation, чтобы выйти на площадь кухни
for i in range(len(fragmentation)):
    if "кухни" in fragmentation[i]:
        area_kit_not_num = fragmentation[i+1] #площадь кухни в не числовом формате

In [93]:
area_kit_not_num

'\n\n17\xa0'

In [94]:
#Преобразование площади кухни к float
if ',' in area_kit_not_num:
    area_kit_not_num = area_kit_not_num.replace(',','.')
try:
    area_kit = float(area_kit_not_num)
except:
    area_kit = '-'

In [95]:
area_kit

17.0

In [96]:
#функция, которая проделывает приведенные выше действия
def Kitsp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    
    fragmentation = re.split('Площадь|:|м2', table)
    
    for i in range(len(fragmentation)):
        if "кухни" in fragmentation[i]:
            area_kit_not_num = fragmentation[i+1] #площадь кухни в не числовом формате
    
            if ',' in area_kit_not_num:
                area_kit_not_num = area_kit_not_num.replace(',','.')
                try:
                    return float(area_kit_not_num)
                except:
                    return '-'
            try:
                return float(area_kit_not_num)
            except:
                return '-'

In [97]:
 Kitsp(flat_page)

17.0

In [98]:
#Cписок под хранение площади кухни
flatStats['Kitsp'] = []
flatStats['Kitsp'] += [Kitsp(flat_page)]

# Данные о продаваемых квартирах (для одного окуруга)

## Функция как рабочий инструмент, который позволяет получать эти данные

In [99]:
#эта функция выполняет все приведенные выше действия для каждой из квартир, из каждого округа Москвы
#N_pages -- число рассматриваемых страниц с квартирами
def get_DATA_district(data,N_pages=30):
    
    #ЦАО
    if data == 'CAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 0
    
    #САО
    if data =='SAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B1%5D=24&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 1
    
    #СВАО
    if data == 'SVAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B1%5D=40&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 2
    
    #ВАО
    if data == 'VAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 3
    
    #ЮВАО
    if data == 'UVAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B10%5D=82&district%5B11%5D=83&district%5B1%5D=73&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 4
    
    #ЮАО
    if data == 'UAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B1%5D=85&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 5
        
    #ЮЗАО
    if data == 'UZAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B10%5D=110&district%5B11%5D=111&district%5B1%5D=101&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 6
    
    #ЗАО
    if data == 'ZAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B1%5D=113&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 7
   
    #СЗАО
    if data == 'SZAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 8
    
    #НАО
    if data == 'NAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=327&district%5B10%5D=337&district%5B1%5D=328&district%5B2%5D=329&district%5B3%5D=330&district%5B4%5D=331&district%5B5%5D=332&district%5B6%5D=333&district%5B7%5D=334&district%5B8%5D=335&district%5B9%5D=336&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 9
    
    #ТАО
    if data == 'TAO':
        district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=338&district%5B1%5D=339&district%5B2%5D=340&district%5B3%5D=341&district%5B4%5D=342&district%5B5%5D=343&district%5B6%5D=344&district%5B7%5D=345&district%5B8%5D=346&district%5B9%5D=347&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
        #числовое обозначение для округа
        district_encode = 10
        
    #Ссылки на квартиры
    links = []
    
    #Вытаскиваем ссылки на описание квартир
    for page in range(1, N_pages): #количество квартир по каждому округу пропорционально параметру N_pages
        page_url =  district.format(page)
        
        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))
        
        for link in flat_urls:
            if link.isdigit():
                links.append(link)
            
    flatStats['District']= []
    flatStats['Price'] = []
    flatStats['Rooms'] = []
    flatStats['Totsp'] = []
    flatStats['Kitsp'] = []
    flatStats['Livesp'] = []
    flatStats['Dist'] = [] 
    flatStats['Metrdist'] = [] 
    flatStats['Walk'] = []
    flatStats['Floor'] = [] 
    flatStats['Nfloors'] = [] 
    flatStats['New'] = []
    flatStats['Bal'] = []
    flatStats['Tel'] = []
    flatStats['Brick'] = []

    for i in range(len(links)):
        flat_url = 'http://www.cian.ru/sale/flat/' + str(links[i]) + '/'
        flat_page = requests.get(flat_url)
        flat_page = flat_page.content
        flat_page = BeautifulSoup(flat_page, 'lxml')
    
        flatStats['District'] += [district_encode]
        flatStats['Price'] += [getPrice(flat_page)]
        flatStats['Rooms'] += [getRoom(flat_page)]
        flatStats['Totsp'] += [Totsp(flat_page)]
        flatStats['Kitsp'] += [Kitsp(flat_page)]
        flatStats['Livesp'] += [Livesp(flat_page)]
        flatStats['Dist'] += [Dist(flat_page)] 
        flatStats['Metrdist'] += [Metrdist(flat_page)] 
        flatStats['Walk'] += [Walk(flat_page)]
        flatStats['Floor'] +=[getFloor(flat_page)] 
        flatStats['Nfloors'] +=[getNfloors(flat_page)] 
        flatStats['New'] += [is_New(flat_page)]
        flatStats['Bal'] += [is_there_Bal(flat_page)]
        flatStats['Tel'] += [is_there_Tel(flat_page)]
        flatStats['Brick'] += [is_Brick(flat_page)]
        
    #Данные возвращаются в формате DataFrame
    return pd.DataFrame(flatStats, columns=['District','Price','Rooms','Totsp','Livesp','Kitsp','Brick','Floor','Nfloors','Dist','Metrdist','Walk','Bal','Tel','New']) #в нумеровнии строк проблема

В этой функции аргумент data может принимать следующие значения:

'CAO', 'SAO', 'SVAO', 'VAO', 'UVAO','UAO','UZAO','ZAO','SZAO','NAO','TAO' -- сокращённое названия округов на латинице

# Данные о продаваемых квартирах со всех округов Москвы 


In [100]:
CAO = get_DATA_district('CAO')
SAO = get_DATA_district('SAO')
SVAO = get_DATA_district('SVAO')
VAO = get_DATA_district('VAO')
UVAO = get_DATA_district('UVAO')
UAO = get_DATA_district('UAO')
UZAO = get_DATA_district('UZAO')
ZAO = get_DATA_district('ZAO')
SZAO = get_DATA_district('SZAO')
NAO = get_DATA_district('NAO')
TAO = get_DATA_district('TAO') 

In [101]:
#Объединение данных по каждому округу
INFO_about_selling_flats = pd.concat([CAO, SAO, SVAO, VAO, UVAO, UAO, UZAO, ZAO, SZAO, NAO, TAO],ignore_index=True)

#Порядковые номера квартиры
N = pd.DataFrame(np.arange(1, len(INFO_about_selling_flats)+1), columns =['N'])
#Добавим их в данные о квартирах
INFO_about_selling_flats = pd.concat([N,INFO_about_selling_flats],axis = 1)

# Наши данные о квартирах Москвы

In [102]:
INFO_about_selling_flats.head(10)

,N,District,Price,Rooms,Totsp,Livesp,Kitsp,Brick,Floor,Nfloors,Dist,Metrdist,Walk,Bal,Tel,New
0,1,0,35584000,2,70.0,43.9,8,1,4,6,1.09,5,1,0,-,0
1,2,0,49000000,3,70.0,40.5,10,1,8,9,1.32,7,1,1,-,1
2,3,0,241316340,5,200.6,-,-,1,3,5,4.57,6,1,0,1,1
3,4,0,4585315,1,17.3,17.3,-,0,2,-,4.74,6,1,0,-,0
4,5,0,4814581,1,17.3,17.3,-,1,2,6,4.77,5,1,0,0,1
5,6,0,5496232,1,23.7,23.7,-,1,2,6,4.77,5,1,0,0,1
6,7,0,5500000,1,32.0,20,7,0,1,9,4.88,4,1,0,1,1
7,8,0,5550000,1,37.4,20,9.6,0,1,1,6.26,10,1,1,-,1
8,9,0,5999000,1,30.0,16,6,1,3,5,9.20,-,0,1,1,1
9,10,0,6000000,1,32.0,20,6,1,5,5,7.70,10,1,0,-,1


In [103]:
INFO_about_selling_flats.tail(10)

,N,District,Price,Rooms,Totsp,Livesp,Kitsp,Brick,Floor,Nfloors,Dist,Metrdist,Walk,Bal,Tel,New
8922,8923,10,3490000,2,61.7,60.2,-,1,3,3,66.66,30,0,1,-,0
8923,8924,10,3500000,1,42.3,16,10,1,1,4,40.18,20,0,1,0,1
8924,8925,10,3500000,2,55.0,36,11,1,2,5,36.42,-,0,1,-,0
8925,8926,10,3500000,2,44.1,37,6.5,0,5,5,49.15,-,0,1,1,1
8926,8927,10,3600000,2,54.0,28,8,0,5,5,60.72,-,0,1,0,1
8927,8928,10,3600000,1,43.0,16,12,1,2,4,40.37,-,0,1,0,1
8928,8929,10,3600000,1,42.0,20,10,1,7,16,32.67,-,0,1,-,0
8929,8930,10,3600000,1,42.3,16,10,1,4,4,40.24,30,0,1,-,1
8930,8931,10,3600000,3,105.0,55,18,1,1,6,36.40,-,0,1,0,1
8931,8932,10,3650000,1,36.0,20,10,0,2,12,33.99,-,0,0,0,1


In [104]:
INFO_about_selling_flats["Totsp"].isnull().sum() #для всех квартир есть информация об общей площади

0

# Cохраняем данные в формате сsv

In [105]:
INFO_about_selling_flats.to_csv("parsing.csv", index = False) #здесь Вы должны указать свой путь, куда Вы хотите сохранить файл 